In [17]:
import os
from openpyxl import load_workbook
import pandas as pd


In [18]:
file_path = "/home/bbc8731/HSV/6_drug_repurposing/data/20260209 DrugsVirusInfo_HSV1-2 Drugs.xlsx"
output_path = "/home/bbc8731/HSV/6_drug_repurposing/data/"



wb = load_workbook(file_path, data_only=True)
ws = wb["Selected HSV1-2 Drugs"]

green_drugs = []

for row in ws.iter_rows(min_row=1):
    cell = row[0]  # Column A = Drug name
    
    fill = cell.fill
    if (
        fill
        and fill.patternType == 'solid'
        and fill.fgColor.type == 'theme'
        and fill.fgColor.theme == 9   # green theme
    ):
        if cell.value and "Green:" not in str(cell.value):  # exclude legend row
            green_drugs.append(cell.value)

df = pd.DataFrame(sorted(set(green_drugs)), columns=["Green Highlighted Drugs"])

In [19]:
from openpyxl import load_workbook
import pandas as pd

file_path = "/home/bbc8731/HSV/6_drug_repurposing/data/20260209 DrugsVirusInfo_HSV1-2 Drugs.xlsx"

wb = load_workbook(file_path, data_only=True)
ws = wb["Selected HSV1-2 Drugs"]

# ---- 1. Detect header row automatically ----
header_row = None
for row in ws.iter_rows():
    values = [str(c.value).strip() if c.value else "" for c in row]
    if "Drug" in values and "Drug_Bank_ID" in values:
        header_row = row[0].row
        headers = values
        break

if header_row is None:
    raise ValueError("Header row not found.")

col_index = {name: headers.index(name) for name in headers}

# ---- 2. Extract green-highlighted rows ----
records = []

for row in ws.iter_rows(min_row=header_row + 1):
    drug_cell = row[col_index["Drug"]]
    
    fill = drug_cell.fill
    
    if (
        fill
        and fill.patternType == "solid"
        and fill.fgColor.type == "theme"
        and fill.fgColor.theme == 9
    ):
        if drug_cell.value:
            records.append({
                "Drug": drug_cell.value,
                # "Virus": row[col_index["Virus"]].value,
                "Drug_Bank_ID": row[col_index["Drug_Bank_ID"]].value,
                "Approval Status": row[col_index["Approval Status"]].value
            })

df = pd.DataFrame(records).drop_duplicates().reset_index(drop=True)

In [21]:
#1. Insert DrugBank ID
df.loc[df["Drug"] == "Cepharanthine", "Drug_Bank_ID"] = "DB16824"

#2. Remove rows with missing Drug_Bank_ID or Approval Status
df_clean = (
    df
    .replace(["NA", "N/A", "na", "None", None, ""], pd.NA)
    .dropna(subset=["Drug_Bank_ID", "Approval Status"])
    .reset_index(drop=True)
)

df_clean.to_csv(os.path.join(output_path, "drugs4validation.csv"), index = False)

In [23]:
# Build formatted lines
lines = [
    f"{row.Drug_Bank_ID}\t# {row.Drug}"
    for _, row in df_clean.iterrows()
]

# Save to text file

with open(os.path.join(output_path,"drugs4validation.txt"), "w") as f:
    f.write("\n".join(lines))

print(f"Saved to {output_path}")


Saved to /home/bbc8731/HSV/6_drug_repurposing/data/
